In [1]:
#Extracting Labels from the labels file
labels_file = "trec06p-cs280/labels"

labels_dict = {}

with open(labels_file, 'r') as file:
    for line in file:
        label, file_path = line.strip().split()
        cleaned_path = file_path.replace('../data/', '').replace('\\', '/')
        labels_dict[cleaned_path] = label


labels_dict

{'000/000': 'ham',
 '000/001': 'spam',
 '000/002': 'spam',
 '000/003': 'ham',
 '000/004': 'spam',
 '000/005': 'ham',
 '000/006': 'ham',
 '000/007': 'spam',
 '000/008': 'spam',
 '000/009': 'spam',
 '000/010': 'ham',
 '000/011': 'spam',
 '000/012': 'spam',
 '000/013': 'spam',
 '000/014': 'spam',
 '000/015': 'spam',
 '000/016': 'spam',
 '000/017': 'spam',
 '000/018': 'spam',
 '000/019': 'spam',
 '000/020': 'ham',
 '000/021': 'ham',
 '000/022': 'spam',
 '000/023': 'spam',
 '000/024': 'ham',
 '000/025': 'ham',
 '000/026': 'ham',
 '000/027': 'ham',
 '000/028': 'spam',
 '000/029': 'spam',
 '000/030': 'spam',
 '000/031': 'spam',
 '000/032': 'ham',
 '000/033': 'spam',
 '000/034': 'ham',
 '000/035': 'ham',
 '000/036': 'spam',
 '000/037': 'spam',
 '000/038': 'ham',
 '000/039': 'ham',
 '000/040': 'spam',
 '000/041': 'spam',
 '000/042': 'spam',
 '000/043': 'spam',
 '000/044': 'ham',
 '000/045': 'ham',
 '000/046': 'ham',
 '000/047': 'ham',
 '000/048': 'spam',
 '000/049': 'ham',
 '000/050': 'spam',
 

In [2]:
#READ AND COMBINE LABELS + EMAILS
import os
from sklearn.model_selection import train_test_split


base_dir = "trec06p-cs280/data"

data = []


for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file:
            # Construct the relative path (e.g., 000/000)
            relative_path = os.path.relpath(os.path.join(root, file), base_dir).replace("\\", "/")

            dir_part = relative_path.split("/")[0]
            
            if dir_part.isdigit():  # Ensure the directory is a number
                dir_number = int(dir_part)
                
                # Read the email content
                with open(os.path.join(root, file), 'r', encoding='latin-1', errors='ignore') as f:
                    content = f.read()
                
                # Get the label for this file from labels_dict
                label = labels_dict.get(relative_path)
                
                if label:
                    # Append the content, label, and directory number
                    data.append((content, label, dir_number))
                
data.sort(key=lambda x: x[2])

emails = [email for email, label, dir_number in data]
labels = [label for email, label, dir_number in data]
directory_numbers = [dir_number for email, label, dir_number in data]

#Partition train set and test sets
train_emails, test_emails, train_labels, test_labels, train_dirs, test_dirs = train_test_split(
    emails, labels, directory_numbers, test_size=0.44, shuffle=False)  # 44% for test as directories 71–127

print("Training Set:\n")
for email, label in zip(train_emails[:5], train_labels[:5]):
    train_data = list(zip(train_emails, train_labels)) #Train data combined emails and labels of the training set
    print(f"Label: {label}, Email: {email[:60]}...\n")


print("\nTest Set:\n")
for email, label in zip(test_emails[:5], test_labels[:5]):
    print(f"Label: {label}, Email: {email[:60]}...\n")



#Displaying first 5 emails and their labels
#Tuple organization: EMAIL , LABEL
# for email, label in data[:5]:
#     print(f"Label: {label}, Email: {email[:60]}...\n")



Training Set:

Label: ham, Email: Received: from rodan.UU.NET by aramis.rutgers.edu (5.59/SMI4...

Label: spam, Email: Received: from unknown (HELO groucho.cs.psu.edu) ([222.135.2...

Label: spam, Email: Received:
	from 24-151-178-89.dhcp.kgpt.tn.charter.com (HELO...

Label: ham, Email: Received: from psuvax1.cs.psu.edu ([130.203.2.4]) by groucho...

Label: spam, Email: Received: from 201-1-198-159.dsl.telesp.net.br (HELO 32A3F2F...


Test Set:

Label: ham, Email: Received: from cereberos.shinigami.org (IDENT:root@detroit1-...

Label: spam, Email: Received: from psy1.psych.arizona.edua (unknown [222.160.107...

Label: spam, Email: Received: from wonder.hananet.net (unknown [219.255.79.54])
...

Label: spam, Email: Received: from media.mit.edu (unknown [85.158.73.138]) by al...

Label: spam, Email: Received: from 43458748 (210.211.253.213.bb-dynamic.vsnl.net...



In [ ]:
#Check if such file exist in a certain set
def find_directory_set():
    while True:
        user_input = input("Enter a directory number (e.g., '001') or type 'exit' to quit: ").strip()
        
        if user_input.lower() == 'exit':
            print("Exiting.")
            break
        
        # Ensure the input is numeric and has three digits
        if not user_input.isdigit() or len(user_input) != 3:
            print("Invalid input. Please enter a three-digit directory number.")
            continue
        
        dir_number = int(user_input)
        
        if dir_number in train_dirs:
            print(f"Directory '{user_input}' found in the training set.")
        elif dir_number in test_dirs:
            print(f"Directory '{user_input}' found in the test set.")
        else:
            print(f"Directory '{user_input}' not found in the dataset.")

# Example usage
find_directory_set()

In [3]:
#DATA CLEANING
import email
from email import policy
from email.parser import BytesParser

def extract_email_body(email_content):
    # Parse the email content into a message object
    message = email.message_from_string(email_content, policy=policy.default)

    email_body = ""

    # If the email is multipart (has multiple parts like plain text or HTML)
    if message.is_multipart():
        for part in message.iter_parts():
            # Look for a text/plain part and ignore any text/html parts
            if part.get_content_type() == "text/plain":
                email_body = part.get_payload(decode=True).decode('latin-1', errors='replace')
                break
    else:
        # If it's not multipart, we just decode the payload (assuming it's text)
        email_body = message.get_payload(decode=True).decode('latin-1', errors='replace')

    # This removes HTML tags using a regex
    import re
    email_body = re.sub(r'<.*?>', '', email_body)

    return email_body



def load_stop_words(file_path):
    with open(file_path, 'r') as f:
        stop_words = {word.strip() for word in f.readlines()}
    return stop_words

def clean_data(text, stop_words):
    # Remove punctuation and replace with space
    words = text.split()  # Split text into words
    cleaned_words = []
    
    for word in words:
        # Remove punctuation from each word
        cleaned_word = ''.join([char if char not in punctuation and char.isalpha() else ' ' for char in word])
        
        # If the cleaned word is not a stop word or a single letter except i and a, append it. Otherwise, replace with space
        if len(cleaned_word) > 1 and cleaned_word.lower() not in stop_words:
            cleaned_words.append(cleaned_word)
        # else:
        #     cleaned_words.append('')  
    
    # Join the cleaned words back into a single string
    cleaned_text = ' '.join(cleaned_words)
    
    return cleaned_text.strip()

stop_words = load_stop_words('stop_words.txt')
processed_data = []
punctuation = '!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'


for raw_email, label, _ in data[:5]:  # we use "_" to ignore dir_number because dir number is for data split only
    email_body = extract_email_body(raw_email)
    cleaned_email = clean_data(email_body,stop_words)
    processed_data.append((cleaned_email,label))

for email_body, label in processed_data[:5]:
    print(f"Label: {label}, Email Body: {email_body[:60]}...\n")
    


Label: ham, Email Body: mailing list queried weeks ago running  set archive server  ...

Label: spam, Email Body: LUXURY WATCHES BUY ROLEX       Rolex    Cartier    Bvlgari  ...

Label: spam, Email Body: Academic Qualifications prestigious NON ACC REDITED uni vers...

Label: ham, Email Body: Greetings all  verify subscription plan  fans list  charter ...

Label: spam, Email Body: chauncey conferred luscious continued tonsillitis...



In [4]:
#To list 10000 unique words

#Dictionaries to count word occurrences for spam and ham
word_count_spam = {}
word_count_ham = {}

def tokenize_email(email_content):
    words = email_content.split()

    return [word for word in words if len(word) > 2]

for raw_email, label in train_data: 
    email_body = extract_email_body(raw_email)
    cleaned_email = clean_data(email_body, stop_words)

    words = tokenize_email(cleaned_email)

    if label == 'spam':
        for word in words:
            if word in word_count_spam:
                word_count_spam[word] += 1
            else:
                word_count_spam[word] = 1

    elif label == 'ham':
        for word in words:
            if word in word_count_ham:
                word_count_ham[word] += 1
            else:
                word_count_ham[word] = 1

unique_words = set(word_count_spam.keys()).union(set(word_count_ham.keys()))

word_occurrences = []
for word in unique_words:
    total_count = word_count_spam.get(word, 0) + word_count_ham.get(word, 0)
    word_occurrences.append((word, total_count, word_count_spam.get(word, 0), word_count_ham.get(word, 0)))

word_occurrences.sort(key=lambda x: x[1], reverse=True)
top_10000_words_withcount = word_occurrences[:10000]

for word, total, spam_count, ham_count in top_10000_words_withcount[:10]:
    print(f"Word: {word}, Total: {total}, Spam: {spam_count}, Ham: {ham_count}")

Word: http, Total: 17397, Spam: 13021, Ham: 4376
Word: com, Total: 17012, Spam: 11808, Ham: 5204
Word: will, Total: 10691, Spam: 4733, Ham: 5958
Word: edu, Total: 8928, Spam: 306, Ham: 8622
Word: www, Total: 7867, Spam: 4002, Ham: 3865
Word: nbsp, Total: 6889, Spam: 5641, Ham: 1248
Word: board, Total: 4178, Spam: 236, Ham: 3942
Word: Adobe, Total: 4050, Spam: 4034, Ham: 16
Word: From, Total: 3995, Spam: 47, Ham: 3948
Word: CRUST, Total: 3989, Spam: 0, Ham: 3989


# Forming Feature Matrice

In [5]:
#get the words only
top_10000_words = [word for word, _, _, _ in word_occurrences[:10000]]

import numpy as np

def generate_feature_matrix(emails, word_list):
    """
    Generate a feature matrix for emails based on the presence of words in word_list.
    """
    matrix = np.zeros((len(emails), len(word_list)), dtype=int)
    
    for i, raw_email in enumerate(emails):
        email_body = extract_email_body(raw_email)
        cleaned_email = clean_data(email_body, stop_words)
        # Tokenize the cleaned email
        words = set(tokenize_email(cleaned_email))
        
        # Update the matrix with 1 if the word exists in the email
        for j, word in enumerate(word_list):
            if word in words:
                matrix[i, j] = 1
    
    return matrix

# Generate the training and testing feature matrices
train_features = generate_feature_matrix(train_emails, top_10000_words)
test_features = generate_feature_matrix(test_emails, top_10000_words)

print("Training Feature Matrix Shape:", train_features.shape)
print("Testing Feature Matrix Shape:", test_features.shape)

Training Feature Matrix Shape: (21180, 10000)
Testing Feature Matrix Shape: (16642, 10000)


# Computing the Priors

In [6]:
# Count the number of spam and ham emails
N_spam = train_labels.count('spam')
N_ham = train_labels.count('ham')

# Total number of emails in the training set
N_doc = len(train_labels)

# Compute the priors
P_spam = N_spam / N_doc
P_ham = N_ham / N_doc

print(f"P(c = spam) = {P_spam}")
print(f"P(c = ham) = {P_ham}")

P(c = spam) = 0.6461756373937677
P(c = ham) = 0.3538243626062323


# Computing for the Likelihood of each word

In [10]:
import numpy as np
import email
from email import policy
from email.parser import BytesParser


# Count word occurrences for spam and ham
word_count_spam = {}
word_count_ham = {}

for raw_email, label in train_data:
    email_body = extract_email_body(raw_email)
    cleaned_email = clean_data(email_body, stop_words)
    words = tokenize_email(cleaned_email)
    
    if label == "spam":
        for word in words:
            word_count_spam[word] = word_count_spam.get(word, 0) + 1
    elif label == "ham":
        for word in words:
            word_count_ham[word] = word_count_ham.get(word, 0) + 1

# Vocabulary
vocabulary = set(word_count_spam.keys()).union(set(word_count_ham.keys()))
vocab_size = len(vocabulary)

# Total word counts in spam and ham
total_spam_words = sum(word_count_spam.values())
total_ham_words = sum(word_count_ham.values())

# Laplace smoothing parameter
lambda_ = 1

# Compute likelihoods for each word
likelihood_spam = {}
likelihood_ham = {}

for word in vocabulary:
    # P(w|spam)
    likelihood_spam[word] = (word_count_spam.get(word, 0) + lambda_) / (total_spam_words + vocab_size * lambda_)
    # P(w|ham)
    likelihood_ham[word] = (word_count_ham.get(word, 0) + lambda_) / (total_ham_words + vocab_size * lambda_)

# Display some examples
for word in list(vocabulary)[:10]:  # Show first 10 words
    print(f"Word: {word}, P(w|spam): {likelihood_spam[word]}, P(w|ham): {likelihood_ham[word]}")

Word: CHARME, P(w|spam): 9.756192743343837e-07, P(w|ham): 5.793167372881356e-06
Word: dhunsche, P(w|spam): 9.756192743343837e-07, P(w|ham): 1.6551906779661016e-06
Word: Hultman, P(w|spam): 9.756192743343837e-07, P(w|ham): 2.4827860169491524e-06
Word: AbZAtsG, P(w|spam): 9.756192743343837e-07, P(w|ham): 2.4827860169491524e-06
Word: demark, P(w|spam): 2.9268578230031513e-06, P(w|ham): 8.275953389830508e-07
Word: ÙÅËÊÎÂÑÊÀß, P(w|spam): 2.9268578230031513e-06, P(w|ham): 8.275953389830508e-07
Word: STABILITY, P(w|spam): 2.9268578230031513e-06, P(w|ham): 8.275953389830508e-07
Word: DgAAACUDBQC, P(w|spam): 9.756192743343837e-07, P(w|ham): 1.6551906779661016e-06
Word: Geometrically, P(w|spam): 9.756192743343837e-07, P(w|ham): 1.6551906779661016e-06
Word: AAAKBwAA, P(w|spam): 9.756192743343837e-07, P(w|ham): 1.6551906779661016e-06


# Classfying the emails and Implementing on "Test_emails"

In [16]:
import numpy as np
import email
from email import policy
from email.parser import BytesParser

# Example priors (from earlier computation)
prior_spam = N_spam / N_doc
prior_ham = N_ham / N_doc

# Log of priors
log_prior_spam = np.log(prior_spam)
log_prior_ham = np.log(prior_ham)

# Function to classify an email
def classify_email(email_content, likelihood_spam, likelihood_ham, log_prior_spam, log_prior_ham):
    # Preprocess email
    email_body = extract_email_body(email_content)
    cleaned_email = clean_data(email_body, stop_words)
    words = tokenize_email(cleaned_email)
    
    # Compute log probabilities
    log_prob_spam = log_prior_spam
    log_prob_ham = log_prior_ham
    
    for word in words:
        log_prob_spam += np.log(likelihood_spam.get(word, 1 / (total_spam_words + vocab_size)))
        log_prob_ham += np.log(likelihood_ham.get(word, 1 / (total_ham_words + vocab_size)))
    
    # Classification
    return "spam" if log_prob_spam > log_prob_ham else "ham"


# Test email classification
correct_predictions = 0
total_emails = len(test_emails)


print("Classifying Test Emails:\n")
for i, raw_email in enumerate(test_emails):
    predicted_label = classify_email(raw_email, likelihood_spam, likelihood_ham, log_prior_spam, log_prior_ham)
    actual_label = test_labels[i]
    
    # Check if prediction matches the actual label
    if predicted_label == actual_label:
        correct_predictions += 1
    
    print(f"Email {i+1}:")
    print(f"Content: {raw_email[:60]}...")
    print(f"Predicted: {predicted_label}, Actual: {actual_label}\n")

# Calculate accuracy
accuracy = correct_predictions / total_emails * 100
print(f"Accuracy: {accuracy:.2f}%")


Classifying Test Emails:

Email 1:
Content: Received: from cereberos.shinigami.org (IDENT:root@detroit1-...
Predicted: ham, Actual: ham

Email 2:
Content: Received: from psy1.psych.arizona.edua (unknown [222.160.107...
Predicted: spam, Actual: spam

Email 3:
Content: Received: from wonder.hananet.net (unknown [219.255.79.54])
...
Predicted: spam, Actual: spam

Email 4:
Content: Received: from media.mit.edu (unknown [85.158.73.138]) by al...
Predicted: spam, Actual: spam

Email 5:
Content: Received: from 43458748 (210.211.253.213.bb-dynamic.vsnl.net...
Predicted: spam, Actual: spam

Email 6:
Content: Received: from gwsmtp.thomson-csf.com (gwsmtp.thomson-csf.co...
Predicted: ham, Actual: ham

Email 7:
Content: Received:
	from unlabelled-14.87.174.82.alphyra.versatel.net...
Predicted: spam, Actual: spam

Email 8:
Content: Received: from mailserver.hmtmtm.com (mailserver.hmtmtm.com ...
Predicted: spam, Actual: spam

Email 9:
Content: Received: from CEYLAN.dz3eo.org (unknown [88.224.168.87]

In [17]:
# Calculate accuracy
accuracy = correct_predictions / total_emails * 100
print(f"Accuracy: {accuracy:.2f}%")

Accuracy: 95.64%


# Performing Evaluation on the classify_email() function

In [18]:
# Initialize counters
true_positive = 0
true_negative = 0
false_positive = 0
false_negative = 0

# Evaluate each email
for i, email_content in enumerate(test_emails):
    # Get the actual and predicted labels
    actual_label = test_labels[i]
    predicted_label = classify_email(email_content, likelihood_spam, likelihood_ham, log_prior_spam, log_prior_ham)
    
    # Update counts based on comparison
    if actual_label == "spam" and predicted_label == "spam":
        true_positive += 1
    elif actual_label == "ham" and predicted_label == "ham":
        true_negative += 1
    elif actual_label == "ham" and predicted_label == "spam":
        false_positive += 1
    elif actual_label == "spam" and predicted_label == "ham":
        false_negative += 1

# Calculate metrics
accuracy = (true_positive + true_negative) / (true_positive + true_negative + false_positive + false_negative)
recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) != 0 else 0
precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) != 0 else 0

# Print the metrics
print(f"Evaluation Metrics:")
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")

# Optional: Print the confusion matrix
print("\nConfusion Matrix:")
print(f"True Positive (TP): {true_positive}")
print(f"True Negative (TN): {true_negative}")
print(f"False Positive (FP): {false_positive}")
print(f"False Negative (FN): {false_negative}")

Evaluation Metrics:
Accuracy: 95.64%
Recall: 96.60%
Precision: 96.93%

Confusion Matrix:
True Positive (TP): 10844
True Negative (TN): 5072
False Positive (FP): 344
False Negative (FN): 382
